# Поиск токсичных комментариев

<a name="1"></a>
## Содержание

[1. Содержание](#1)

[2. Описание проекта](#2)

*    [2.1. Цель проекта](#21)
*    [2.2. Задачи проекта](#22)
*    [2.3. Описание данных](#23)
*    [2.4. План работы](#24)

[3. Подготовка данных](#3)

*    [3.1. Изучение данных](#31)
*    [3.2. Сэмплирование данных](#32)
*    [3.3. Разделение сэмпла на выборки](#33)
*    [3.4. Получение эмбеддингов (с *BERT*)](#34)
*    [3.5. Вывод](#35)

[4. Обучение и тестирование моделей](#4)

*    [4.1. Функция для обучения моделей](#41)
*    [4.2. `LogisticRegression`](#42)
*    [4.3. `RandomForestClassifier`](#43)
*    [4.4. `LGBMClassifier`](#44)
*    [4.5. `CatBoostClassifier`](#45)
*    [4.6. Сравнение моделей](#46)
*    [4.7. Тестирование лучшей модели](#47)
*    [4.8. Вывод](#48)

[5. Общий вывод](#5)

## Описание проекта
<a name="2"></a> <div style="text-align: left">[Cодержание](#Содержание)</div>

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75.

### Цель проекта
<a name="21"></a> <div style="text-align: left">[Cодержание](#Содержание)</div>

Проведение исследования с целью построения модели машинного обучения, которая поможет классифицировать комментарии на позитивные и негативные.

Результаты исследования позволят магазину искать токсичные комментарии и отправлять их на модерацию.

### Задачи проекта
<a name="22"></a> <div style="text-align: left">[Cодержание](#Содержание)</div>

1. Загрузить и изучить данные.
2. Сэмплировать данные.
3. Разделить сэмплы на выборки.
4. Получить эмбеддинги.
5. Построить и обучить модели.
6. Протестировать лучшую модель.
7. Написать общий вывод.

Решим поставленную в проекте задачу, применяя **нейронную сеть *BERT***.

Построим модель со значением метрики качества *F1* не меньше 0.75.

### Описание данных
<a name="23"></a> <div style="text-align: left">[Cодержание](#Содержание)</div>

В нашем распоряжении набор данных с разметкой о токсичности правок.

Данные находятся в файле `/datasets/toxic_comments.csv`.

Столбец `text` в нём содержит текст комментария, а `toxic` - целевой признак.

### План работы
<a name="24"></a> <div style="text-align: left">[Cодержание](#Содержание)</div>

1. Загрузим и изучим данные.
2. Сэмплируем данные: возьмём из датасета выборку из 1000 случайных элементов.
3. Разделим сэмплированную выборку на обучающую и тестовую выборки в соотношении 4:1.
4. Переведём текст комментариев в векторные представления (эмбеддинги) на базе нейронной сети *BERT*. Используем библиотеки  `torch` и `transformers`.
7. Обучим 4 модели: `LogisticRegression`, `RandomForestClassifier`, `LGBMClassifier` и `CatBoostClassifier` с различными гиперпараметрами.
8. Сравненим модели.
9. Протестируем лучшую модель и напишем вывод.

## Подготовка данных
<a name="3"></a> <div style="text-align: left">[Cодержание](#Содержание)</div>

In [3]:
import numpy as np
import pandas as pd
import torch
import transformers
import os
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score, \
GridSearchCV, KFold, train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tqdm import notebook

RANDOM_STATE = 12345
TEST_SIZE = 0.2

### Изучение данных
<a name="31"></a> <div style="text-align: left">[Cодержание](#Содержание)</div>

Загрузим данные.

In [4]:
pth1 = r'C:/Users/Васильев/Desktop/dataframes/toxic_comments.csv'
pth2 = '/datasets/toxic_comments.csv'
pth3 = '/toxic_comments.csv'
pth4 = 'https://code.s3.yandex.net/datasets/toxic_comments.csv'
pth5 = '/content/toxic_comments.csv'

if os.path.exists(pth1):
    data = pd.read_csv(pth1)
elif os.path.exists(pth2):
    data = pd.read_csv(pth2)
elif os.path.exists(pth3):
    data = pd.read_csv(pth3)
elif os.path.exists(pth4):
    data = pd.read_csv(pth4)
elif os.path.exists(pth5):
    data = pd.read_csv(pth5)
else:
    print('Something is wrong')

In [5]:
display(data.sample(5))

,Unnamed: 0,text,toxic
136873,137011,"""\n\nThe user Goldorack keeps on reverting the...",0
58870,58936,http://catalogue.statelibrary.tas.gov.au/find/...,0
2808,2808,Scott's boss whom he almost killed was called ...,0
158193,158352,Is this drug really that expensive? You can g...,0
6447,6451,Explain on their Talk page that they need to u...,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Видно, что загруженный датасет имеет большой размер.

In [7]:
print(data.shape)

(159292, 3)


In [8]:
data.describe()

,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000


Проверим данные на наличие явных дубликатов.

In [9]:
data.duplicated().sum()

0

Посчитаем количество классов в таргете.

In [10]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Вывод нам показывает сильный дисбаланс классов в таргете.

### Сэмплирование данных
<a name="32"></a> <div style="text-align: left">[Cодержание](#Содержание)</div>

Будем решать поставленную в проекте задачу с помощью **нейронной сети *BERT*** (от англ. *Bidirectional Encoder Representations from Transformers*, «двунаправленная нейронная сеть-кодировщик») - нейронная сеть для создания модели языка.

Чтобы машины воспринимали слова, картинки или аудио, их преобразовывают в векторный вид. Когда работают с текстом, его тоже переводят в векторный формат, или векторные представления. Частный случай этих представлений - *word embeddings* (англ. «слова-вложения»; «эмбеддинги»). Работают они так: сложная структура (текст) вкладывается в более простую - вектор.
Векторы-эмбеддинги содержат данные о соотношении разных слов и их свойствах. Привычное понимание свойства слова, его смысла и контекста справедливо и для машинного обучения.

Чтобы не создавать эмбеддинги слишком долго, возьмём из датасета выборку из 1000 случайных элементов.

In [11]:
data_sample = data.sample(
    n=1000, random_state=RANDOM_STATE).reset_index(drop=True)

Проверим, сохранилось ли в таргете сэмплированной выборки соотношение количества классов.

In [12]:
data_sample['toxic'].value_counts()

0    890
1    110
Name: toxic, dtype: int64

### Разделение сэмпла на выборки
<a name="33"></a> <div style="text-align: left">[Cодержание](#Содержание)</div>

Разделим сэмплированную выборку `data_sample` на обучающую и тестовую, в соотношении 4:1.

In [13]:
data_sample_train, data_sample_test, target_train, target_test = train_test_split(
    data_sample['text'], data_sample['toxic'], test_size=TEST_SIZE,
    stratify=data_sample['toxic'], random_state=RANDOM_STATE)

print('Размеры выборок:')
print(f"train - {len(data_sample_train)} \
- {len(data_sample_train)/len(data_sample['text']):.0%}")

print(f"test - {len(data_sample_test)} \
- {len(data_sample_test)/len(data_sample['text']):.0%}")

Размеры выборок:
train - 800 - 80%
test - 200 - 20%


### Получение эмбеддингов (с *BERT*)
<a name="34"></a> <div style="text-align: left">[Cодержание](#Содержание)</div>

Переведём текст комментариев в векторные представления, также известные как эмбеддинги, с помощью **нейронной сети *BERT***. 

Для этого используем модель [unitary/toxic-bert](https://huggingface.co/unitary/toxic-bert/tree/main), предобученную специально для идентификации токсичных комментариев. У *BERT* имеется собственный токенизатор. Лемматизация для *BERT* не требуется.

Решим задачу проекта на *PyTorch* (англ. «факел для Python»). Библиотека `torch` применяется в задачах обработки естественного текста и компьютерного зрения. А нам нужна для работы с моделью *BERT*, которая находится в библиотеке `transformers` (англ. «трансформеры»).

Для получения эмбеддингов, напишем функцию `make_embeddings()`.

In [129]:
def make_embeddings(data_sample):

    # Инициализируем токенизатор как объект класса 'BertTokenizer()'.
    # Передадим ему файл со словарём, на котором обучалась модель. 
    # Он может быть, например,в текстовом формате (txt). 
    # Зададим длину текстов 'max_length=512'.
    tokenizer = transformers.BertTokenizer.from_pretrained('vocab.txt', max_length=512)

    # Токенизируем комментарии:
    # Преобразуем текст в номера токенов из словаря методом 'encode()'
    # (англ. «закодировать»). Для корректной работы модели мы указали
    # аргумент 'add_special_tokens=True'. Это значит, что к любому преобразуемому
    # тексту добавляется токен начала (101) и токен конца текста (102).
    tokenized = data_sample.apply(
        (lambda x: tokenizer.encode(x, add_special_tokens=True)))

    # Применим метод padding (англ. «отступ»), чтобы после токенизации
    # длины исходных текстов в корпусе были равными - только при таком 
    # условии будет работать модель BERT. Стандартная длина вектора
    # будет 'max_len=512', остальные векторы дополним нулями.
    max_len = 512
    padded = np.array([i + [0]*(max_len - len(i)) if len(i)<512 \
                       else i[:512] for i in tokenized.values])

    # Cоздадим маску для выделения важных токенов:
    # Поясним модели, что нули не несут значимой информации. Это нужно
    # для компоненты модели, которая называется «внимание» (англ. attention).
    # Отбросим эти токены и «создадим маску» для действительно важных
    # токенов, то есть укажем нулевые и не нулевые значения.
    attention_mask = np.where(padded != 0, 1, 0)

    # отобразим размеры преобразованных данных
    print(f"tokenized shape: {tokenized.shape}")
    print(f"padded shape: {padded.shape}")
    print(f"attention_mask shape: {attention_mask.shape}")

    # Инициализируем конфигурацию 'BertConfig' (англ. Bert Configuration).
    # В качестве аргумента передадим ей JSON-файл с описанием настроек модели.
    config = transformers.BertConfig.from_json_file('config.json')
    
    # Затем инициализируем саму модель класса 'BertModel'.
    # Передадим ей файл с предобученной моделью и конфигурацией.
    model = transformers.BertModel.from_pretrained('pytorch_model.bin', config=config)
    
    # Модель BERT создаёт эмбеддинги батчами. Чтобы хватило оперативной памяти,
    # сделаем размер батча небольшим - создадим эмбеддинги батчами по 20 текстов.
    batch_size = 20
    # Напишемм цикл по батчам. Отображать прогресс будет функция 'notebook()'.
    # Для хранения эмбеддингов твитов сделаем список 'embeddings'.
    embeddings = []
    for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
            # Преобразуем данные в формат тензоров - многомерных векторов
            # в библиотеке torch. Тип данных 'LongTensor' хранит числа в длинном формате,
            # т.е. выделяет на каждое число по 64 бита.
            batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
            # Преобразуем маску.
            attention_mask_batch = torch.LongTensor(
                attention_mask[batch_size*i:batch_size*(i+1)])

            # Получим эмбеддинги для батча:
            # Для ускорения вычисления функцией 'no_grad()' в библиотеке torch укажем, 
            # что градиенты не нужны - модель BERT обучать не будем.
            with torch.no_grad():
                # чтобы получить эмбеддинги для батча, передадим модели данные и маску
                batch_embeddings = model(batch, attention_mask=attention_mask_batch)

            # Из полученного тензора извлечём нужные элементы и добавим их в список
            # всех эмбеддингов. Преобразуем элементы методом 'numpy()' к типу 'numpy.array'.
            embeddings.append(batch_embeddings[0][:,0,:].numpy())

    # Вызовом функции concatenate() cоберём и выведем все эмбеддинги в матрицу признаков
    return np.concatenate(embeddings)

Применим функцию `make_embeddings()` к обучающей выборке `data_sample_train`, получим эмбеддинги `feature_train` - признаки для обучения наших моделей.

In [130]:
%%time

feature_train = make_embeddings(data_sample_train)
print(f"features shape: {feature_train.shape}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1743: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


tokenized shape: (800,)
padded shape: (800, 512)
attention_mask shape: (800, 512)


  0%|          | 0/40 [00:00<?, ?it/s]

features shape: (800, 768)
CPU times: user 26min 36s, sys: 4min 44s, total: 31min 21s
Wall time: 31min 35s


Применим функцию `make_embeddings()` к тестовой выборке `data_sample_test`, получим эмбеддинги `feature_test` - это признаки для тестирования лучшей модели.

In [131]:
%%time

feature_test = make_embeddings(data_sample_test)
print(f"features shape: {feature_test.shape}")

tokenized shape: (200,)
padded shape: (200, 512)
attention_mask shape: (200, 512)


  0%|          | 0/10 [00:00<?, ?it/s]

features shape: (200, 768)
CPU times: user 6min 37s, sys: 720 ms, total: 6min 38s
Wall time: 6min 42s


### Вывод
<a name="35"></a> <div style="text-align: left">[Cодержание](#Содержание)</div>

В разделе [**Подготовка данных**](#Подготовка-данных) были выполнены следующие задачи:
1. данные изучены;
2. данные сэмплированы - из датасета взята выборка `data_sample` из 1000 случайных элементов;
3. сэмплированная выборка разделена на обучающую и тестовую выборки, в соотношении 4:1;
4. получены эмбеддинги, для обучения и тестирования.


В результате выполнения задач данного раздела было выявлено следующее:
1. пропуски в данных отсутствуют;
2. типы данных соответствуют их содержанию;
3. датасет имеет большой размер - содержит 159 292 текстовых комментария;
4. явных дубликаты отсутствуют;
5. наблюдается сильный дисбаланс классов в таргете.

**В проекте решается задача бинарной классификации.**

Таким образом, данные подготовлены для обучения моделей.

## Обучение и тестирование моделей
<a name="4"></a> <div style="text-align: left">[Cодержание](#Содержание)</div>

Выполним обучение четырех моделей: логистическую регрессию (*Logistic Regression*), случайный лес (*Random Forest*), градиентные бустинги *LightGBM* и *CatBoost* для задачи классификации. Для обучения логистической регрессии используем функцию `cross_val_score()`, а для последних трёх моделей применим поиск гиперпараметров с помощью функции `GridSearchCV`.

Реализуем учёт баланса классов в модели с помощью параметра `class_weight`.

### Функция для обучения моделей
<a name="41"></a> <div style="text-align: left">[Cодержание](#Содержание)</div>

Напишем функцию `fit_model()` для обучения моделей `RandomForestClassifier`, `LGBMClassifier` и `CatBoostClassifier` и вычисления метрики качества *F1*. Метрика *F1* лучшей модели будет выводиться как `model.best_score_`.

Используем разделение на фолды с помощью *KFold*. Зададим параметры для кроссвалидации - `n_splits`, количество фолдов.

In [132]:
k_fold = KFold(n_splits=3, random_state=RANDOM_STATE, shuffle=True)

In [133]:
def fit_model(estimator, param_grid, feature_train, target_train):
    model = GridSearchCV(estimator=estimator,
                         param_grid=param_grid,
                         cv=k_fold,
                         scoring='f1')

    model.fit(feature_train, target_train)
    best_f1 = round(model.best_score_, 2)

    print(f"Best F1: {best_f1}")
    print(f"Best params: {model.best_params_}")

    return model.best_estimator_, best_f1

### `LogisticRegression`
<a name="42"></a> <div style="text-align: left">[Cодержание](#Содержание)</div>

Классифицируем обучающие данные с помощью модели логистической регрессии ***Logistic Regression*** - `LogisticRegression`. Для автоматизации предварительных преобразований данных перед обучением модели используем функцию `cross_val_score()` .

In [134]:
%%time

model_lr = LogisticRegression(random_state=RANDOM_STATE,
                              class_weight='balanced')
model_lr.fit(feature_train, target_train)

f1_lr = cross_val_score(model_lr, feature_train, target_train,
                        scoring='f1', cv=4).mean()

print('F1:', f'{f1_lr:.2f}')

F1: 0.93
CPU times: user 681 ms, sys: 369 ms, total: 1.05 s
Wall time: 612 ms


На обучающей выборке, модель логистической регрессии **`LogisticRegression`**  имеет следующее значение метрики оценки качества:
- ***F1 = 0.93***

### `RandomForestClassifier`
<a name="43"></a> <div style="text-align: left">[Cодержание](#Содержание)</div>

Построим модель случайного леса ***Random Forest*** - `RandomForestClassifier`.

In [135]:
%%time

rf_estimator = RandomForestClassifier(random_state=RANDOM_STATE,
                                      class_weight='balanced')

rf_param_grid = {
    # количество деревьев
    'n_estimators': list(range(60, 121, 30)),
    # максимальная глубина дерева
    'max_depth': list(range(2, 13, 5)),
}

rf_best_model = fit_model(estimator=rf_estimator,
                          param_grid=rf_param_grid,
                          feature_train=feature_train,
                          target_train=target_train)

Best F1: 0.94
Best params: {'max_depth': 7, 'n_estimators': 60}
CPU times: user 11 s, sys: 87.7 ms, total: 11 s
Wall time: 11.1 s


На обучающей выборке, лучшая модель случайного леса **`RandomForestClassifier`** имеет следующее значение метрики оценки качества:
- ***F1 = 0.94***

при следующих гиперпараметрах:
- глубина дерева: `max_depth` = 7
- количество деревьев: `n_estimators` = 60

### `LGBMClassifier`
<a name="44"></a> <div style="text-align: left">[Cодержание](#Содержание)</div>

Построим модель градиентного бустинга ***LightGBM*** - `LGBMClassifier`.

In [136]:
%%time

lgbm_estimator = LGBMClassifier(random_state=RANDOM_STATE,
                                class_weight='balanced')

lgbm_param_grid = {
    # количество деревьев (итераций)
    "n_estimators": range(25, 101, 25),
    # максимальная глубина дерева
    "max_depth": range(5, 16, 5),
    # коэффициент скорости обучения (размер шага градиентного спуска)
    'learning_rate': [0.15, 0.2, 0.25]
}

lgbm_best_model = fit_model(estimator=lgbm_estimator,
                            param_grid=lgbm_param_grid,
                            feature_train=feature_train,
                            target_train=target_train)

Best F1: 0.94
Best params: {'learning_rate': 0.2, 'max_depth': 10, 'n_estimators': 50}
CPU times: user 3min, sys: 381 ms, total: 3min 1s
Wall time: 3min 1s


На обучающей выборке, лучшая модель градиентного бустинга **`LGBMClassifier`** имеет следующее значение метрики оценки качества:
- ***F1 = 0.94***

при следующих гиперпараметрах:
- глубина дерева: `max_depth` = 10
- количество деревьев: `n_estimators` = 50
- коэффициент скорости обучения (размер шага градиентного спуска): `learning_rate` = 0.2

### `CatBoostClassifier`
<a name="45"></a> <div style="text-align: left">[Cодержание](#Содержание)</div>

Построим модель градиентного бустинга ***CatBoost*** - `CatBoostClassifier`.

Здесь нужно отметить, что `CatBoostClassifier` создает временные файлы для обучения. 

Укажем в модели параметр `allow_writing_files=False`, чтобы это предотвратить и не появлялась ошибка о том, что по указанными путям у `CatBoostClassifier` нет доступа к созданию файлов.

In [137]:
%%time

classes = np.unique(target_train)
weights = compute_class_weight(class_weight='balanced',
                               classes=classes,
                               y=target_train)
class_weights = dict(zip(classes, weights))

catboost_estimator = CatBoostClassifier(random_state=RANDOM_STATE,
                                        verbose=False,
                                        class_weights=class_weights,
                                        allow_writing_files=False)


catboost_param_grid = {
    # количество итераций
    "iterations": range(100, 201, 50),
    # глубина дерева
    "depth": range(2, 7, 2),
    # коэффициент скорости обучения (размер шага градиентного спуска)
    "learning_rate": [0.1, 0.15, 0.2]}

catboost_best_model = fit_model(estimator=catboost_estimator,
                                param_grid=catboost_param_grid,
                                feature_train=feature_train,
                                target_train=target_train)

Best F1: 0.92
Best params: {'depth': 4, 'iterations': 200, 'learning_rate': 0.15}
CPU times: user 37min 58s, sys: 13.4 s, total: 38min 11s
Wall time: 23min 27s


На обучающей выборке, лучшая модель градиентного бустинга **`CatBoostClassifier`** имеет следующее значение метрики оценки качества:
- ***F1 = 0.92***

при следующих параметрах:
- глубина дерева: `depth` = 4
- количество итераций: `iterations` = 200
- коэффициент скорости обучения (размер шага градиентного спуска): `learning_rate` = 0.15

### Сравнение моделей
<a name="46"></a> <div style="text-align: left">[Cодержание](#Содержание)</div>

В виде таблицы `table`, выведем значения метрики *F1* разных моделей на обучающей выборке.

In [138]:
table = pd.DataFrame([
     ['LogisticRegression', f'{f1_lr:.2f}'],
     ['RandomForestClassifier', f'{rf_best_model[1]:.2f}'],
     ['LGBMClassifier', f'{lgbm_best_model[1]:.2f}' ],
     ['CatBoostClassifier', f'{catboost_best_model[1]:.2f}']
    ],
columns=['model', 'F1'])

print('F1 для разных моделей')
table

F1 для разных моделей


,model,F1
0,LogisticRegression,0.93
1,RandomForestClassifier,0.94
2,LGBMClassifier,0.94
3,CatBoostClassifier,0.92


В качестве лучшей, выберем модель градиентного бустинга **`LGBMClassifier`**, со значением метрики ***F1 = 0.94***.

### Тестирование лучшей модели
<a name="47"></a> <div style="text-align: left">[Cодержание](#Содержание)</div>

Проверим на тестовой выборке качество выбранной модели градиентного бустинга **`LGBMClassifier`**.

In [139]:
f1_lgbm_test = f1_score(target_test, lgbm_best_model[0].predict(feature_test))
print('F1 на тестовой выборке:', f'{f1_lgbm_test:.2f}')

F1 на тестовой выборке: 0.95


На тестовой выборке, лучшая модель градиентного бустинга **`LGBMClassifier`** имеет следующее значение метрики оценки качества:
- ***F1 = 0.95***

Видно, что по сравнению с обучающей выборкой, качество на тестовой не ухудшилось.

Также, значение метрики *F1* на тестовой выборке 0.95 > 0.75, что соответствует требованию в условии задачи проекта.

### Вывод
<a name="48"></a> <div style="text-align: left">[Cодержание](#Содержание)</div>

В разделе [**Обучение и тестирование моделей**](#Обучение-и-тестирование-моделей) были выполнены следующие задачи:
1. написана функция `fit_model()` для обучения и вычисления метрики *F1* для моделей с использованием `GridSearchCV`;
2. обучено четыре модели: `LogisticRegression`, `RandomForestClassifier`, `LGBMClassifier` и `CatBoostClassifier` с различными гиперпараметрами;
3. выбрана лучшая модель, её качество проверено на тестовой выборке.

В результате выполнения задач этого раздела было выявлено следующее:
1. В качестве лучшей выбрана модель **`LGBMClassifier`**, у которой на обучающей выборке значение метрики оценки качества ***F1 = 0,94***.
2. Лучшая модель градиентного бустинга **`LGBMClassifier`** на тестовой выборке имеет следующее значение метрики оценки качества:
- ***F1 = 0.95***

По сравнению с обучающей выборкой, качество лучшей модели на тестовой выборке не ухудшилось.

Значение метрики *F1* на тестовой выборке превышает 0.75, что соответствует требованию в условии задачи проекта.

## Общий вывод
<a name="5"></a> <div style="text-align: left">[Cодержание](#Содержание)</div>

Нами проведено исследование, с целью построения модели машинного обучения, которая поможет классифицировать комментарии на позитивные и негативные. 

Поставленная задача была решена с помощью **нейронной сети *BERT***.

Входные данными был набор данных с разметкой о токсичности правок.

В разделе [**подготовки данных**](#Подготовка-данных) данные сначала были изучены, было выявлено, что:

1. пропуски в данных отсутствовали;
2. типы данных соответствовали их содержанию;
3. датасет имел большой размер и содержал 159 292 текстовых комментария;
4. явных дубликаты отсутствовали;
5. наблюдался сильный дисбаланс классов в таргете.

После изучения, данные были сэмплированы, взята выборка из 1000 случайных элементов. Далее, сэмплированная выборка разделена на обучающую и тестовую выборки в пропорции 4:1. Затем создана функция, благодаря которой были получены эмбеддинги, для обучения и тестирования. Таким образом, данные были подготовлены для обучения моделей.

В разделе [**обучения и тестирования моделей**](#Обучение-и-тестирование-моделей) перед обучением моделей была написана функция для обучения и вычисления метрики *F1* - для моделей с использованием `GridSearchCV`. Затем были обучены четыре модели: `LogisticRegression`, `RandomForestClassifier`, `LGBMClassifier` и `CatBoostClassifier` с различными гиперпараметрами.


В ходе исследования удалось получить следующие результаты **на обучающей выборке**:


1. Модель логистической регрессии **`LogisticRegression`** имеет следующее значение метрики оценки качества:
    - ***F1 = 0.93***


2. Лучшая модель случайного леса **`RandomForestClassifier`** имеет следующее значение метрики оценки качества:
    - ***F1 = 0.94***

   при следующих параметрах:
    - глубина дерева: `max_depth` = 7
    - количество деревьев: `n_estimators` = 60


3. Лучшая модель градиентного бустинга **`LGBMClassifier`** имеет следующее значение метрики оценки качества:
    - ***F1 = 0.94***

   при следующих параметрах:
    - глубина дерева: `max_depth` = 10
    - количество деревьев: `n_estimators` = 50
    - коэффициент скорости обучения (размер шага градиентного спуска): `learning_rate` = 0.2


4. Лучшая модель градиентного бустинга **`CatBoostClassifier`** имеет следующее значение метрики оценки качества:
    - ***F1 = 0.92***

   при следующих параметрах:
    - глубина дерева: `depth` = 4
    - количество итераций: `iterations` = 200
    - коэффициент скорости обучения (размер шага градиентного спуска): `learning_rate` = 0.15
    

После обучения всех моделей, была создана [**сводная таблица**](#Сравнение-моделей) и в качестве лучшей выбрана модель **`LGBMClassifier`**, у которой на обучающей выборке значение метрики оценки качества ***F1 = 0,94***.

Затем качество лучшей модели было [**проверено на тестовой выборке**](#Тестирование-лучшей-модели). Лучшая модель градиентного бустинга **`LGBMClassifier`** на тестовой выборке имеет значение метрики оценки качества ***F1 = 0.95***

По сравнению с обучающей выборкой, качество лучшей модели на тестовой выборке не ухудшилось.

_____________________ ______________________ ______________________      
Исходя из полученных результатов, можно сделать следующие **выводы**:


1. В качестве лучшей модели выбрана модель **градиентного бустинга `LGBMClassifier`**, которая **на тестовой выборке** имеет следующее значение метрики оценки качества  - ***F1 = 0.95***

   
2. Для выбранной лучшей модели значение метрики качества ***F1* превышает 0.75**, что соответсвует требованию в условии задачи проекта.
   
______________________ ______________________ ______________________   
**Общие рекомендации:**

Магазину можно рекомендовать использовать полученную модель **`LGBMClassifier`** в качестве инструмента, который будет искать токсичные комментарии и отправлять их на модерацию.

Результаты исследования позволят магазину искать токсичные комментарии и отправлять их на модерацию.
______________________ ______________________ ______________________
На этом мы заканчиваем и благодарим вас за внимание. Надеемся, что наше исследование вам понравилось и поможет сделать правильные выводы, принять правильные решения.